In [2]:
import pandas as pd
import numpy as np
xmas = pd.read_csv("https://www.dropbox.com/scl/fi/qxaslqqp5p08i1650rpc4/xmas.csv?rlkey=erdxi7jbh7pqf9fh4lv4cayp5&dl=1")
xmas

,Day,Day.in.Words,Gift.Item,Verb,Adjective,Location
0,1,first,partridge,NaN,NaN,in a pear tree
1,2,second,dove,NaN,turtle,NaN
2,3,third,hen,NaN,french,NaN
3,4,fourth,bird,NaN,calling,NaN
4,5,fifth,ring,NaN,golden,NaN
5,6,sixth,goose,a-laying,NaN,NaN
6,7,seventh,swan,a-swimming,NaN,NaN
7,8,eighth,maid,a-milking,NaN,NaN
8,9,ninth,lady,dancing,NaN,NaN
9,10,tenth,lord,a-leaping,NaN,NaN


#### Function 1: pluralize_gift() <br>
The gifts are listed in singular: for example, on day five the narrator receives “five golden rings”, but the entry in the data set for the gift on day five simply says “ring”. <br>
Using the skeleton of the pluralize_gift() function, complete the code so that the function takes a gift and returns the appropriate plural.

In [3]:
def pluralize_gift(gift):
    if gift.endswith('y'):
        gift = gift[:-1] + 'ies'
    elif 'oo' in gift:
        gift = gift.replace('oo', 'ee')
    else: 
        gift += 's'
    return gift
    

In [4]:
pluralize_gift = np.vectorize(pluralize_gift)
pluralize_gift(xmas['Gift.Item'])

array(['partridges', 'doves', 'hens', 'birds', 'rings', 'geese', 'swans',
       'maids', 'ladies', 'lords', 'pipers', 'drummers'], dtype='<U10')

#### Function 2: make_phrase() <br>
Write a function called make_phrase() that takes as input the necessary information, and returns a phrase.

In [41]:
# The main function
def make_phrase(day, day_word, gift, verb, adjective, location):
    """
    Constructs the phrase for the '12 Days of Christmas' song format.
    """

    ## Step 1: Replace NAs with blank strings
    verb = verb if verb != "nan" else ''
    adjective = adjective if adjective != "nan" else ''
    location = location if location != "nan" else ''
    map_day =  {
    'first': 'one',
    'second': 'two',
    'third': 'three',
    'fourth': 'four',
    'fifth': 'five',
    'sixth': 'six',
    'seventh': 'seven',
    'eighth': 'eight',
    'ninth': 'nine',
    'tenth': 'ten',
    'eleventh': 'eleven',
    'twelfth': 'twelve'}

    word_day = map_day.get(day_word)
    ## Step 2: If the day number is larger than 1, the gift items need pluralized!
    if day > 1:
        gift = pluralize_gift(gift)

    ## Step 3: Figure out if a gift item starts with a vowel
    if isinstance(gift, str) and len(gift) > 0:
        vowels = ['a', 'e', 'i', 'o', 'u']
        bvowel = gift[0].lower() in vowels  # Use .lower() with parentheses
    else: 
        bvowel = False
    ## Step 4: For the first day, handle the 'a' or 'an' condition
    if day == 1:
        if bvowel:
            day = 'an'
        else:
            day = 'a'
    else:
        day = word_day

    ## Step 5: Construct the final phrase and return it
    phrase = f"{day} {adjective} {gift} {verb} {location}".strip()
    return phrase


Test Your Function
Make sure to try your function out on small examples and on the xmas data.

In [42]:
make_phrase(day = 1, day_word = "first", gift = "snake", verb = "fed", adjective = "red", location = "garden")

'a red snake fed garden'

In [43]:
make_phrase(day = 2, day_word = "second", gift = "snake", verb = "fed", adjective = "red", location = "garden")

'two red snakes fed garden'

In [44]:
xmas['Verb'] = xmas['Verb'].astype(str)
xmas['Adjective'] = xmas['Adjective'].astype(str)
xmas['Location'] = xmas['Location'].astype(str)

xmas['Verb'] = xmas['Verb'].fillna('')
xmas['Adjective'] = xmas['Adjective'].fillna('')
xmas['Location'] = xmas['Location'].fillna('')

Then, use the function to make a new column of the xmas column called Full.Phrase containing the sentences for the new gift on that day.

In [45]:
# Apply the function and print the results
xmas['Full Phrase'] = xmas.apply(
    lambda row: make_phrase(
        day = row["Day"],
        day_word = row["Day.in.Words"],
        gift = row["Gift.Item"],
        verb = row["Verb"],
        adjective = row["Adjective"],
        location = row["Location"]
    ),
    axis = 1
)

xmas['Full Phrase'].fillna('', inplace = True)
print(xmas[[ "Full Phrase"]])

                     Full Phrase
0   a  partridge  in a pear tree
1               two turtle doves
2              three french hens
3             four calling birds
4              five golden rings
5            six  geese a-laying
6        seven  swans a-swimming
7         eight  maids a-milking
8           nine  ladies dancing
9           ten  lords a-leaping
10         eleven  pipers piping
11     twelve  drummers drumming
                     Full Phrase
0   a  partridge  in a pear tree
1               two turtle doves
2              three french hens
3             four calling birds
4              five golden rings
5            six  geese a-laying
6        seven  swans a-swimming
7         eight  maids a-milking
8           nine  ladies dancing
9           ten  lords a-leaping
10         eleven  pipers piping
11     twelve  drummers drumming


<positron-console-cell-45>:14: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.




#### Function 3: sing_day() <br>
Write a function called sing_day() that takes as input:

A dataset (input as a dataframe)

A number indicating which day to sing about (input as an integer)

The name of a column in the dataset that contains the phrases for each day (input as an tidy name)

In [72]:
def sing_day(dataset, num, phrase_col):
  """
  <documentation>
  """
  
  # Step 1: Setup the intro line
  map_num_to_word = {
    1:'first',
    2:'second',
    3:'third',
    4:'fourth',
    5:'fifth',
    6:'sixth',
    7:'seventh',
    8:'eighth',
    9:'ninth',
    10:'tenth',
    11:'eleventh',
    12:'twelvth'}

  num_word = map_num_to_word.get(num)  # convert "1" to "first" etc.
  intro = "On the " + num_word + " day of Christmas, my true love sent to me:"
  
  # Step 2: Sing the gift phrases
  # Hint: What order are they gifts sung in each day?
  gifts = ""
  for i in range(num, 0, -1):
    if i == 1 and num != 1:
      gifts += "and " + dataset.loc[dataset['Day'] == i, phrase_col].values[0] + "\n"
    else: 
      gifts += dataset.loc[dataset['Day'] == i, phrase_col].values[0] + "\n"

  # Step 3: Put it all together and return
  return intro + "\n" + gifts.strip()

#### Test your function <br>
Use this code to show the function worsk:

In [73]:
sing_day(xmas, 3, "Full Phrase")


'On the third day of Christmas, my true love sent to me:\nthree french hens\ntwo turtle doves\nand a  partridge  in a pear tree'

Use Your Functions!
Run appropriate code to output the lyrics for the entire 12 Days of Christmas song.

In [80]:
full_song = []
for k in range(1,13): 
    line = sing_day(xmas, k, "Full Phrase")
    full_song.append(line)

In [81]:
for i, line in enumerate(full_song): 
    if i > 0:
        print("\n")
    print(line)

On the first day of Christmas, my true love sent to me:
a  partridge  in a pear tree


On the second day of Christmas, my true love sent to me:
two turtle doves
and a  partridge  in a pear tree


On the third day of Christmas, my true love sent to me:
three french hens
two turtle doves
and a  partridge  in a pear tree


On the fourth day of Christmas, my true love sent to me:
four calling birds
three french hens
two turtle doves
and a  partridge  in a pear tree


On the fifth day of Christmas, my true love sent to me:
five golden rings
four calling birds
three french hens
two turtle doves
and a  partridge  in a pear tree


On the sixth day of Christmas, my true love sent to me:
six  geese a-laying
five golden rings
four calling birds
three french hens
two turtle doves
and a  partridge  in a pear tree


On the seventh day of Christmas, my true love sent to me:
seven  swans a-swimming
six  geese a-laying
five golden rings
four calling birds
three french hens
two turtle doves
and a  partr

Then, load the following dataset, and run your code again on this dataset instead to get a surprise song! (The column names and formats of xmas2 are the same as those for xmas.)

In [83]:
xmas2 = pd.read_csv("https://www.dropbox.com/scl/fi/p9x9k8xwuzs9rhp582vfy/xmas_2.csv?rlkey=kvc3j3lmyn4opcidsrhcmrof1&dl=1")
xmas2

,Day,Day.in.Words,Gift.Item,Verb,Adjective,Location
0,1,first,email,NaN,NaN,from Cal Poly
1,2,second,point,NaN,meal,NaN
2,3,third,pen,NaN,lost,NaN
3,4,fourth,review,NaN,course,NaN
4,5,fifth,exam,NaN,practice,NaN
5,6,sixth,grader,grading,NaN,NaN
6,7,seventh,senior,stressing,NaN,NaN
7,8,eighth,mom,a-calling,NaN,NaN
8,9,ninth,party,bumping,NaN,NaN
9,10,tenth,load,of laundry,NaN,NaN


In [113]:
xmas2['Verb'] = xmas2['Verb'].astype(str)
xmas2['Adjective'] = xmas2['Adjective'].astype(str)
xmas2['Location'] = xmas2['Location'].astype(str)

xmas2['Verb'] = xmas2['Verb'].fillna('')
xmas2['Adjective'] = xmas2['Adjective'].fillna('')
xmas2['Location'] = xmas2['Location'].fillna('')

xmas2['Full Phrase'] = xmas2.apply(
    lambda row: make_phrase(
        day = row["Day"],
        day_word = row["Day.in.Words"],
        gift = row["Gift.Item"],
        verb = row["Verb"],
        adjective = row["Adjective"],
        location = row["Location"]
    ),
    axis = 1
)

xmas2['Full Phrase'].fillna('', inplace = True)


<positron-console-cell-113>:21: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.




In [116]:
full_song2 = []
for k in range(1,13): 
    line = sing_day(xmas2, k, "Full Phrase")
    full_song2.append(line)


In [117]:
for i, line in enumerate(full_song2): 
    if i > 0:
        print("\n")
    print(line)

On the first day of Christmas, my true love sent to me:
an  email  from Cal Poly


On the second day of Christmas, my true love sent to me:
two meal points
and an  email  from Cal Poly


On the third day of Christmas, my true love sent to me:
three lost pens
two meal points
and an  email  from Cal Poly


On the fourth day of Christmas, my true love sent to me:
four course reviews
three lost pens
two meal points
and an  email  from Cal Poly


On the fifth day of Christmas, my true love sent to me:
five practice exams
four course reviews
three lost pens
two meal points
and an  email  from Cal Poly


On the sixth day of Christmas, my true love sent to me:
six  graders grading
five practice exams
four course reviews
three lost pens
two meal points
and an  email  from Cal Poly


On the seventh day of Christmas, my true love sent to me:
seven  seniors stressing
six  graders grading
five practice exams
four course reviews
three lost pens
two meal points
and an  email  from Cal Poly


On the e